<a href="https://colab.research.google.com/github/PlayfulDevBit/Grafana-influx/blob/main/Grafana.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install psutil influxdb-client

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.6/65.6 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 746.2/746.2 kB 17.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 217.8/217.8 kB 13.3 MB/s eta 0:00:00


In [11]:
# Import necessary libraries
import psutil
import time
from influxdb_client import InfluxDBClient, Point, WritePrecision
from influxdb_client.client.write_api import SYNCHRONOUS
from google.colab import userdata # Import userdata to access secrets

# --- Configuration ---

# Fetch credentials from Colab Secrets
# IMPORTANT: Make sure you have created secrets in Colab:
#   - Name: TOKEN_IDB, Value: Your InfluxDB API Token
#   - Name: ORG_IDB, Value: Your InfluxDB Organization Name or ID
try:
    TOKEN = userdata.get('TOKEN_IDB')
    ORG = userdata.get('ORG_IDB')
except Exception as e:
    print(f"Error accessing Colab secrets. Make sure 'google.colab' is available and secrets are set: {e}")
    # Optionally exit or raise the exception if secrets are critical
    raise SystemExit("Could not retrieve secrets.")


# Check if secrets were retrieved successfully
if not TOKEN:
    raise ValueError("Secret 'TOKEN_IDB' not found or is empty. Please set it in Colab Secrets (View -> Secrets).")
if not ORG:
    raise ValueError("Secret 'ORG_IDB' not found or is empty. Please set it in Colab Secrets (View -> Secrets).")

# InfluxDB connection details (Bucket and URL can remain here or also be moved to secrets if desired)
BUCKET = "cpu_metrics"  # Replace with your bucket name if different
URL = "https://eu-central-1-1.aws.cloud2.influxdata.com"  # Replace with your region URL if different

# --- Main Script ---

print("Attempting to connect to InfluxDB...")
print(f"Using Org: '{ORG[:4]}...' (from secret ORG_IDB)") # Print partial org for confirmation
print(f"Using Token: '{TOKEN[:4]}...{TOKEN[-4:]}' (from secret TOKEN_IDB)") # Print partial token for confirmation

# Initialize InfluxDB client using credentials from secrets
try:
    client = InfluxDBClient(url=URL, token=TOKEN, org=ORG)
    write_api = client.write_api(write_options=SYNCHRONOUS)
    # Optional: Add a check to verify connection (e.g., ping)
    # health = client.health()
    # if health.status == "pass":
    #    print(f"InfluxDB connection successful to org '{health.org}'!")
    # else:
    #    print(f"InfluxDB connection check failed: {health.message}")
    #    raise ConnectionError("Failed InfluxDB health check")


except Exception as e:
    print(f"Error initializing InfluxDB Client: {e}")
    print("Please check your URL and ensure the fetched Org ID and Token are correct and have permissions.")
    raise SystemExit("InfluxDB client initialization failed.")


print(f"Successfully initialized client. Writing data to bucket '{BUCKET}'...")

# Collect CPU usage for 60 seconds
try:
    for i in range(12):  # Collect data every 5 seconds for 60 seconds
        cpu_percent = psutil.cpu_percent(interval=1) # Measures over 1 sec

        # Create a data point
        point = Point("system_metrics") \
                .tag("host", "colab") \
                .field("cpu_usage", cpu_percent) \
                .time(time.time_ns(), WritePrecision.NS)

        # Write to InfluxDB
        # The bucket and org are specified here again, which is good practice for clarity
        write_api.write(bucket=BUCKET, org=ORG, record=point)
        print(f"[{i+1}/12] Sent CPU Usage: {cpu_percent}%")

        # Wait approx 5 seconds total including measurement time
        time.sleep(4)

except Exception as e:
    print(f"\nAn error occurred during data writing: {e}")
    print("This might be due to network issues, bucket permissions, or server-side problems.")
finally:
    # Close the client gracefully regardless of errors
    print("\nClosing InfluxDB client.")
    if 'client' in locals() and client:
        client.close()

print("Script finished.")

Attempting to connect to InfluxDB...
Using Org: 'Toni...' (from secret ORG_IDB)
Using Token: '5UoP...mA==' (from secret TOKEN_IDB)
Successfully initialized client. Writing data to bucket 'cpu_metrics'...
[1/12] Sent CPU Usage: 4.0%
[2/12] Sent CPU Usage: 2.5%
[3/12] Sent CPU Usage: 3.0%
[4/12] Sent CPU Usage: 2.5%
[5/12] Sent CPU Usage: 47.0%
[6/12] Sent CPU Usage: 3.0%
[7/12] Sent CPU Usage: 37.1%
[8/12] Sent CPU Usage: 4.1%
[9/12] Sent CPU Usage: 2.5%
[10/12] Sent CPU Usage: 3.0%
[11/12] Sent CPU Usage: 2.0%
[12/12] Sent CPU Usage: 2.0%

Closing InfluxDB client.
Script finished.
